In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
import torch
from PIL import Image
from torch import nn
import torch
from torchvision import transforms, models
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
class PCNN(nn.Module):
    def __init__(self):
        super(PCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 12 * 12, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 2)  # happy or sad

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x
class SCNN(nn.Module):
    def __init__(self):
        super(SCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(512 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 4)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 100 -> 50
        x = self.pool(F.relu(self.conv2(x)))  # 50 -> 25
        x = self.pool(F.relu(self.conv3(x)))  # 25 -> 12
        x = self.pool(F.relu(self.conv4(x)))  # 12 -> 6
        x = self.pool(F.relu(self.conv5(x)))  # 6 -> 3
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        return self.fc3(x)
class HybridEmotionRecognizer:
    def __init__(self, pcnn, scnn, device='cuda'):
        self.pcnn = pcnn.to(device).eval()
        self.scnn = scnn.to(device).eval()
        self.device = device

        self.sub_emotion_map = {
            0: 'surprise',  # Happy path
            1: 'neutral',
            2: 'angry',     # Sad path
            3: 'fear'
        }

    def predict(self, image_tensor):
        with torch.no_grad():
            image_tensor = image_tensor.to(self.device)

            # P-CNN
            primary_logits = self.pcnn(image_tensor)
            primary_class = torch.argmax(primary_logits, dim=1).item()  # 0 = happy, 1 = sad

            # S-CNN
            sub_logits = self.scnn(image_tensor)
            sub_class = torch.argmax(sub_logits, dim=1).item()

            # Interpret result based on primary emotion
            print("primary_class:", primary_class)
            if primary_class == 0:  # Happy
                label = self.sub_emotion_map[sub_class if sub_class in [0, 1] else 1] # default neutral
                print("the subclass:", sub_class)
            else:  # Sad
                label = self.sub_emotion_map[sub_class if sub_class in [2, 3] else 2] # default anger
                print("the subclass:", sub_class)

        return {
            "primary": "happy" if primary_class == 0 else "sad",
            "secondary": label
        }

In [ ]:
p_model = PCNN()
s_model = SCNN()

p_model.load_state_dict(torch.load("/content/drive/MyDrive/RAF_DB_analysis/saved_models/pcnn_model.pth", map_location=torch.device('cuda')))
s_model.load_state_dict(torch.load("/content/drive/MyDrive/RAF_DB_analysis/saved_models/scnn_model.pth", map_location=torch.device('cuda')))

p_model.eval()
s_model.eval()

SCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=4, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
class ImgDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform or transforms.ToTensor()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

# load dataset
path = "/content/drive/MyDrive/RAF_DB_analysis/AffectNetCustom/test"
image_paths = []
labels = []
for label_str in sorted(os.listdir(path)):
    label_path = os.path.join(path, label_str)
    if not os.path.isdir(label_path):
        continue
    files = glob.glob(os.path.join(label_path, '*.jpg'))
    image_paths.extend(files)
    # Folder name = label ID (0–6)
    labels.extend([int(label_str)] * len(files))
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])
dataset = ImgDataset(image_paths, labels, transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False)


In [ ]:
def evaluate_hybrid(hybrid_model, loader, device="cuda"):
    valid_labels = {
        5: 0,  # surprise
        6: 1,  # neutral
        0: 2,  # angry
        2: 3   # fear
    }
    subclass_map = {
        "surprise": 0,
        "neutral": 1,
        "angry": 2,
        "fear": 3
    }
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            for i in range(images.size(0)):
                label = labels[i].item()
                if label not in valid_labels:
                    continue
                img = images[i].unsqueeze(0).to(device)
                pred = hybrid_model.predict(img)
                expected_subclass = valid_labels[label]
                expected_primary = 0 if expected_subclass in [0, 1] else 1
                pred_subclass = subclass_map.get(pred["secondary"], -1)
                pred_primary = 0 if pred["primary"] == "happy" else 1
                if pred_primary == expected_primary and pred_subclass == expected_subclass:
                    correct += 1
                total += 1
    acc = correct / total if total > 0 else 0
    print(f"Hybrid Test Accuracy (Secondary emotion path only): {acc:.4f} on {total} samples")
    return acc, total

In [ ]:
pipeline = HybridEmotionRecognizer(p_model, s_model)
evaluate_hybrid(pipeline, loader)

primary_class: 0
the subclass: 1
primary_class: 1
the subclass: 3
primary_class: 0
the subclass: 3
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 0
the subclass: 1
primary_class: 0
the subclass: 3
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 2
primary_class: 0
the subclass: 3
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 2
primary_cl

(0.3252032520325203, 1599)

In [ ]:
#transfer learning with Resnet
def build_resnet_finetune(num_classes=7):
    resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    for param in resnet.parameters():
        param.requires_grad = False
    # Replace final layer
    in_features = resnet.fc.in_features
    resnet.fc = nn.Sequential(
        nn.Linear(in_features, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    return resnet

In [ ]:
resnet_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
#read training and testing data
data_dir   = '/content/drive/MyDrive/RAF_DB_analysis/RAF_DB/'

test_dir = data_dir + 'DATASET/test/'
train_dir = data_dir + 'DATASET/train/'

test_csv  = pd.read_csv(data_dir + 'test_labels.csv')
train_csv = pd.read_csv(data_dir + 'train_labels.csv')

In [ ]:
test_label = test_csv['label'].to_list()
test_label = [label - 1 for label in test_label]
test_img = [test_dir+str(r['label'])+'/'+r['image'] for idx, r in test_csv.iterrows()]
train_label = train_csv['label'].to_list()
train_label = [label - 1 for label in train_label]
train_img = [train_dir+str(r['label'])+'/'+r['image'] for idx, r in train_csv.iterrows()]


In [ ]:
dataset_train = ImgDataset(train_img, train_label, resnet_transform)
loader_train = DataLoader(dataset, batch_size=32, shuffle=False)
dataset_test = ImgDataset(test_img, test_label, resnet_transform)
loader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

In [ ]:
def train_resnet(model, train_loader, val_loader, num_epochs=64, lr=1e-5, device="cuda"):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)
    for name, param in model.named_parameters():
        if "layer2" in name or "layer3" in name or "layer4" in name or "fc" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False
    print("Training ResNet...")
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            preds = outputs.argmax(dim=1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        train_acc = train_correct / train_total

        # Validate
        if val_loader:
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item()
                    preds = outputs.argmax(dim=1)
                    val_correct += (preds == labels).sum().item()
                    val_total += labels.size(0)
            val_acc = val_correct / val_total
            print(f"Epoch {epoch+1}/{num_epochs} - "
                f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
                f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        else:
            print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

In [ ]:
def evaluate_resnet(model, test_loader, fer_mapping, device="cuda"):
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu().numpy()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
    acc = correct / total
    print("Test Set Classification Report:\n",
          classification_report(all_labels, all_preds, target_names=list(fer_mapping.keys())))
    print("Test Set Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
    print(f"overall test accuracy: {acc:.4f}")

In [ ]:
resnet_model = build_resnet_finetune()
train_resnet(resnet_model, loader_train, num_epochs= 200, val_loader=None, lr = 1e-5)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 225MB/s]


Training ResNet...
Epoch 1/200 - Train Loss: 184.9673, Train Acc: 0.0486
Epoch 2/200 - Train Loss: 180.2117, Train Acc: 0.0507
Epoch 3/200 - Train Loss: 178.9487, Train Acc: 0.0532
Epoch 4/200 - Train Loss: 177.4681, Train Acc: 0.0514
Epoch 5/200 - Train Loss: 176.6089, Train Acc: 0.0547
Epoch 6/200 - Train Loss: 175.8305, Train Acc: 0.0622
Epoch 7/200 - Train Loss: 175.4280, Train Acc: 0.0629
Epoch 8/200 - Train Loss: 174.7903, Train Acc: 0.0650
Epoch 9/200 - Train Loss: 174.5075, Train Acc: 0.0693
Epoch 10/200 - Train Loss: 174.0180, Train Acc: 0.0779
Epoch 11/200 - Train Loss: 173.5728, Train Acc: 0.0815
Epoch 12/200 - Train Loss: 173.3720, Train Acc: 0.0886
Epoch 13/200 - Train Loss: 173.1078, Train Acc: 0.0997
Epoch 14/200 - Train Loss: 172.9278, Train Acc: 0.0907
Epoch 15/200 - Train Loss: 172.6338, Train Acc: 0.0993
Epoch 16/200 - Train Loss: 172.5420, Train Acc: 0.1043
Epoch 17/200 - Train Loss: 172.3482, Train Acc: 0.0997
Epoch 18/200 - Train Loss: 172.1594, Train Acc: 0.1200


In [ ]:
torch.save(resnet_model.state_dict(), 'resnet_model.pth')

In [ ]:
from google.colab import files
files.download('resnet_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
fer_mapping_raf = {
    'surprise': 0,
    'fear': 1,
    'disgust': 2,
    'happy': 3,
    'sad': 4,
    'angry': 5,
    'neutral': 6
}
evaluate_resnet(resnet_model, loader_test,fer_mapping_raf)

Test Set Classification Report:
               precision    recall  f1-score   support

    surprise       0.00      0.00      0.00       329
        fear       0.12      0.05      0.08        74
     disgust       0.07      0.22      0.10       160
       happy       0.38      0.07      0.12      1185
         sad       0.19      0.08      0.11       478
       angry       0.05      0.15      0.07       162
     neutral       0.24      0.54      0.33       680

    accuracy                           0.18      3068
   macro avg       0.15      0.16      0.12      3068
weighted avg       0.24      0.18      0.15      3068

Test Set Confusion Matrix:
 [[  0   4  55  22  20  70 158]
 [  0   4   7   7   2   7  47]
 [  0   4  35  10   6  26  79]
 [  1   7 208  88  98 203 580]
 [  0   5 100  39  38  69 227]
 [  0   4  17  19   7  24  91]
 [  0   4 105  45  32 125 369]]
overall test accuracy: 0.1819
